# The OAuth 2.0 Authorization Framework

This notebook contains information taken directly from [**The OAuth 2.0 Authorization Framework** (RFC 6749)](https://tools.ietf.org/html/rfc6749). Once I complete this document I'll procceed to describe and explain OAuth 2.0 using my own words.

---

### Introduction

##### Roles

OAuth defines four roles:

* **Resource owner:** An entity capable of granting access to a protected resource. When the resource owner is a person, it is referred to as an *end-user*.
* **Resource server:** The server hosting the protected resource, capable of accepting and responding to protected resource requests using *access tokens*.
* **Client:** An application making protected resource requests on behalf of the resource owner and with its authorization. The term "client" does not imply any particular implementation characteristics (e.g., whether the application executes on a server, a desktop, or other devices).
* **Authorization server:** The server issuing access tokens to the client after successfully authenticating the resource owner and obtaining authorization.

The interaction between the authorization server and resource server is beyond the scope of this specification. The authorization server may be the same server as the resource server or a separate entity. A single authorization server may issue access tokens accepted by multiple resource servers.

##### Protocol Flow

     +--------+                               +---------------+
     |        |--(A)- Authorization Request ->|   Resource    |
     |        |                               |     Owner     |
     |        |<-(B)-- Authorization Grant ---|               |
     |        |                               +---------------+
     |        |
     |        |                               +---------------+
     |        |--(C)-- Authorization Grant -->| Authorization |
     | Client |                               |     Server    |
     |        |<-(D)----- Access Token -------|               |
     |        |                               +---------------+
     |        |
     |        |                               +---------------+
     |        |--(E)----- Access Token ------>|    Resource   |
     |        |                               |     Server    |
     |        |<-(F)--- Protected Resource ---|               |
     +--------+                               +---------------+
     

* (A) The client requests authorization from the resource owner.  The authorization request can be made directly to the resource owner (as shown), or preferably indirectly via the authorization server as an intermediary.
* (B) The client receives an authorization grant, which is a credential representing the resource owner's authorization, expressed using one of four grant types defined in this specification or using an extension grant type. The authorization grant type depends on the method used by the client to request authorization and the types supported by the authorization server.
* (C) The client requests an access token by authenticating with the authorization server and presenting the authorization grant.
* (D) The authorization server authenticates the client and validates the authorization grant, and if valid, issues an access token.
* (E) The client requests the protected resource from the resource server and authenticates by presenting the access token.
* (F) The resource server validates the access token, and if valid, serves the request.

The preferred method for the client to obtain an authorization grant from the resource owner (depicted in steps (A) and (B)) is to **use the authorization server as an intermediary** (...).

##### Access Token\*

Access tokens are credentials used to access protected resources.  An access token is a string representing an authorization issued to the client.  The string is usually opaque to the client.  Tokens represent specific scopes and durations of access, granted by the resource owner, and enforced by the resource server and authorization server.

The token may denote an identifier used to retrieve the authorization information or may self-contain the authorization information in a verifiable manner (i.e., a token string consisting of some data and a signature).  Additional authentication credentials, which are beyond the scope of this specification, may be required in order for the client to use a token.

The access token provides an abstraction layer, replacing different authorization constructs (e.g., username and password) with a single token understood by the resource server.  This abstraction enables issuing access tokens more restrictive than the authorization grant used to obtain them, as well as removing the resource server's need to understand a wide range of authentication methods.

Access tokens can have different formats, structures, and methods of utilization (e.g., cryptographic properties) based on the resource server security requirements.  Access token attributes and the methods used to access protected resources are beyond the scope of this specification and are defined by companion specifications such as [**The OAuth 2.0 Authorization Framework: Bearer Token Usage** (RFC6750)](https://tools.ietf.org/html/rfc6750).

> __\*__ There's a different section about Refresh Tokens

---

### Client Registration